<a href="https://colab.research.google.com/github/poddubnyoleg/nondualbot/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import keras.backend as K
import sys

Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open('/content/drive/My Drive/nondualbot/i_am_that.txt', 'r') 
data = file.read() 

In [5]:
data[:100]

'That which permeates all, which nothing transcends and which, like the universal space around us, fi'

In [0]:
chars = sorted(list(set(data)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [7]:

n_chars = len(data)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  988168
Total Vocab:  55


For each chunk of data [seq_length] we take one-hot-encoded dataX array and dataY array (same but shifted +1)

In [10]:
seq_length = 100

K.clear_session()

model = Sequential()
model.add(LSTM(512, input_shape=(None, n_vocab), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(512, input_shape=(None, n_vocab), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(512, input_shape=(None, n_vocab), return_sequences=True))
model.add(Dropout(0.5))

model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 512)         1163264   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 55)          28215     
Total para

In [0]:
filename = "/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
filepath="/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
num_data = [char_to_int[char] for char in data]

* Inner state doesnt share between samples inside of one batch, but can share between batches, if stateful=True
* fit(shuffle=) doesn't relate to RNNs, because to keep inner state consistent withi epoch, you need to set stateful=True and batch_size=1, so there is nothing to shuffle
* https://stackoverflow.com/questions/43882796/when-does-keras-reset-an-lstm-state




In [0]:
batch_size = 32

def train_generator():
  while True:

    seq_length = np.random.randint(low=2, high=200)
    x_train, y_train = [], []
    
    for _ in range(batch_size):
      
      cur_pos = np.random.randint(low=0, high=n_chars-seq_length-1)

      x_train.append(np_utils.to_categorical(num_data[cur_pos:cur_pos+seq_length], num_classes=n_vocab))
      y_train.append(np_utils.to_categorical(num_data[cur_pos+1:cur_pos+seq_length+1], num_classes=n_vocab))

    yield np.reshape(x_train, (batch_size, seq_length, n_vocab)), np.reshape(y_train, (batch_size, seq_length, n_vocab))


In [16]:
h3 = model.fit_generator(train_generator(), steps_per_epoch=1000, epochs=500, verbose=1, callbacks=callbacks_list)


Epoch 1/500
1000/1000 [==============================] - 587s 587ms/step - loss: 2.2211

Epoch 00001: loss improved from inf to 2.22107, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 2/500
1000/1000 [==============================] - 587s 587ms/step - loss: 1.4822

Epoch 00002: loss improved from 2.22107 to 1.48219, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 3/500
1000/1000 [==============================] - 603s 603ms/step - loss: 1.3465

Epoch 00003: loss improved from 1.48219 to 1.34654, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 4/500
1000/1000 [==============================] - 581s 581ms/step - loss: 1.2981

Epoch 00004: loss improved from 1.34654 to 1.29814, saving model to /content/drive/My Drive/nondualbot/best_current_weights_2.hdf5
Epoch 5/500
1000/1000 [==============================] - 573s 573ms/step - loss: 1.2486

Epoch 00005: loss improved from 1.2

KeyboardInterrupt: ignored

## Create new stateful model with same weights and passing cell state for next prediction
No need to pass all the data from beginning, can feed with 1 example per model.predict, saving state from previous prediction
https://stackoverflow.com/questions/48760472/how-to-use-the-keras-model-to-forecast-for-future-dates-or-events

In [0]:
forecast_model = Sequential()
forecast_model.add(LSTM(512, input_shape=(None, n_vocab), return_sequences=True, 
               stateful=True, batch_input_shape=(1,None,n_vocab)))
forecast_model.add(LSTM(512, input_shape=(None, n_vocab), return_sequences=True, 
               stateful=True))
forecast_model.add(LSTM(512, input_shape=(None, n_vocab), return_sequences=False, 
               stateful=True))

forecast_model.add(Dense(n_vocab, activation='softmax'))

In [0]:
filename = "/content/drive/My Drive/nondualbot/best_current_weights_2.hdf5"
forecast_model.load_weights(filename)
forecast_model.compile(loss='categorical_crossentropy', optimizer='adam')

Predict next letter, save state, predict next letter with saved state

In [24]:
forecast_model.reset_states()

question = 'That which permeates all.'
question_features = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
#question_features = np.append(np.zeros(shape=(max(seq_length-question_features.shape[0],0), n_vocab)), question_features, axis=0)

print(question)

#predict question char by char

sys.stdout.write(' ')

for question_char in question_features:
  question_predicted_char = forecast_model.predict(question_char.reshape(1,1,n_vocab))
  print_char = int_to_char[np.argmax(question_predicted_char)]
  sys.stdout.write(print_char)
  
sys.stdout.write('\n')

for _ in range(150):
  question_predicted_char = forecast_model.predict(question_predicted_char.reshape(1,1,n_vocab))
  print(question_predicted_char)
  print_char = int_to_char[np.argmax(question_predicted_char)]
  sys.stdout.write(print_char)

That which permeates all.
 het ihich irrcaated inl  
[[1.82659132e-04 1.16047137e-04 7.55004585e-05 4.03304771e-03
  1.57337417e-05 6.90649003e-02 6.10863678e-02 1.53085226e-02
  2.83614211e-02 2.31491141e-02 7.55538931e-03 4.36877925e-03
  4.02968787e-02 2.05966532e-01 5.22282906e-03 9.76320647e-04
  9.50149726e-03 1.39622409e-02 1.94170028e-02 3.38643529e-02
  1.26578808e-02 1.12300005e-03 6.90235104e-03 2.85232253e-02
  2.15878353e-01 2.88320333e-03 2.70962861e-04 1.20333463e-01
  6.64151683e-02 2.06209937e-04 8.92081298e-05 2.86197501e-05
  7.10623717e-05 1.64337558e-04 6.56532357e-05 7.34780406e-06
  1.19237746e-04 1.31939189e-04 1.38041751e-05 2.41185462e-05
  1.03359409e-04 9.24553169e-05 8.62886009e-05 1.42085453e-04
  9.50567846e-05 1.07188735e-05 2.04605283e-04 2.02902826e-04
  2.36249994e-04 6.24276290e-05 1.07600863e-05 1.33414316e-04
  9.94534730e-07 1.83858385e-04 5.61516288e-07]]
T[[6.2398938e-04 9.6114827e-06 2.3291655e-05 1.8769313e-05 1.1293067e-06
  5.5455690e-08 4.1

#Old prediction

In [20]:
question = 'That which permeates all.'
question_features = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
question_features = np.append(np.zeros(shape=(max(seq_length-question_features.shape[0],0), n_vocab)), question_features, axis=0)
print(question)

test_prediction = model.predict(question_features.reshape(1,len(question_features),n_vocab))[0]
sys.stdout.write(' ')
for test_char in test_prediction:
  print_char = int_to_char[np.argmax(test_char)]
  sys.stdout.write(print_char)

That which permeates all.
  ees es  aa   aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa   aaaddaaaada      het ihich irrcaated all  

In [21]:
question = 'That which permeates all.'
pattern = np_utils.to_categorical([char_to_int[char] for char in question], num_classes=n_vocab)
pattern = np.append(np.zeros(shape=(max(seq_length-pattern.shape[0],0), n_vocab)), pattern, axis=0)

print(question)

test_prediction = model.predict(pattern.reshape(1,len(pattern),n_vocab))[0]
sys.stdout.write(' ')
for test_char in test_prediction:
  print_char = int_to_char[np.argmax(test_char)]
  sys.stdout.write(print_char)

# generate characters
for i in range(150):

  prediction = model.predict(np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1])), verbose=0)[0][-1]
  
  index = np.random.choice(len(prediction), p=(prediction**2)/sum(prediction**2))#np.argmax(prediction)#
  result = int_to_char[index]
  sys.stdout.write(result)
  pattern = np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)
  pattern = pattern[1:len(pattern)]

That which permeates all.
  ees es  aa   aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa   aaaddaaaada      het ihich irrcaated all   The self is the power of life, which is not perfect. A sense of being is beyond the mind and the universe is not the subject. What can the heart of t

In [24]:
model.predict(np.reshape(pattern, (1, pattern.shape[0], pattern.shape[1])), verbose=0)[0][-1]

array([9.9999940e-01, 5.4691748e-07, 4.8891195e-12, 1.0469446e-09,
       2.6797808e-10, 6.7335869e-22, 2.8095636e-26, 6.3552258e-29,
       7.0487436e-23, 6.3646487e-26, 3.1674805e-26, 4.9144446e-28,
       1.0800542e-21, 1.1273043e-25, 3.6994567e-28, 6.7637692e-34,
       3.3639891e-28, 2.0014950e-27, 2.9739306e-31, 5.9701945e-25,
       2.0236516e-27, 0.0000000e+00, 1.8087869e-34, 1.6505187e-24,
       1.3041807e-22, 2.8014332e-31, 8.8703745e-35, 1.8485967e-19,
       6.2777927e-26, 6.5178172e-19, 1.2295411e-16, 1.3757429e-19,
       6.6819938e-13, 3.0502608e-13, 2.4411573e-12, 2.3483241e-15,
       1.7099024e-17, 1.6124142e-16, 2.0943558e-22, 2.6491773e-17,
       1.4324105e-12, 8.1101988e-14, 7.6933120e-11, 5.4370323e-14,
       4.9228969e-17, 4.9811773e-23, 2.1248222e-15, 8.4897835e-12,
       2.5380966e-16, 1.9712886e-15, 8.8841031e-17, 3.3209028e-15,
       2.6880777e-22, 6.2216196e-20, 2.2106736e-28], dtype=float32)

In [58]:
np.append(pattern, [np_utils.to_categorical(index,num_classes=n_vocab)], axis=0)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
patte

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

In [23]:
np.argmax(dataY[0])

40

In [24]:
dataY[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)